In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

def prepare_data():
    """ 
    準備資料
    Returns:
        X_train(ndarray): 訓練資料 (50000.32.32.3)
        X_test(ndarray): 測試資料 (10000.32.32.3)
        y_train(ndarray): 將訓練資料改為One-hot encoding後的標籤 (50000,10)
        y_train(ndarray): 將測試資料改為One-hot encoding後的標籤 (10000,10)
        y_test_label(ndarray): 測試資料的正確答案 (10000)
    """
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    # 對訓練資料與測試資料的圖像執行常規化
    x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
    x_train, x_test = x_train/255.0, x_test/255.0
    # 將訓練資料與測試資料的標籤轉換為以One-hot encoding來呈現10的類別
    y_train, y_test = to_categorical(y_train), to_categorical(y_test)

    return x_train, x_test, y_train, y_test


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers

def make_convlayer():
    """ 
    建立模型
    """
    # Sequential物件
    model = Sequential()
    # 卷積層 1
    model.add(Conv2D(filters=64, 
                     kernel_size=3, 
                     padding='same',
                     activation='relu', 
                     input_shape=(32,32,3)))
    # 2 × 2 池化層
    model.add(MaxPooling2D(pool_size=2))
    # 卷積層 2
    model.add(Conv2D(filters=128, 
                     kernel_size=3, 
                     padding='same',
                     activation='relu'))
    # 2 × 2 池化層
    model.add(MaxPooling2D(pool_size=2))
    # 卷積層 3
    model.add(Conv2D(filters=256, 
                     kernel_size=3, 
                     padding='same',
                     activation='relu')) 
    #2 × 2 池化層 
    model.add(MaxPooling2D(pool_size=2)) 
    # 扁平層 
    model.add(Flatten()) 
    # 丟棄法
    model.add(Dropout(0.4))
    # 第7層
    model.add(Dense(512, activation='relu'))
    # 輸出層
    model.add(Dense(10, activation='softmax'))

    # 優化器為 Adam
    model.compile(loss="categorical_crossentropy",
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=["accuracy"])
    return model


In [ ]:
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

def train(x_train, x_test, y_train, y_test):
    """
    Parameters:
        x_train, x_test, y_train, y_test: 訓練資料與測試資料
    Returns:
        History object
    """
    # 如果經過2個訓練週期仍未看見val_loss獲得改善、就將學習率減為0.5倍。
    reduce_lr = ReduceLROnPlateau(
        monitor='val_accuracy',    # 監控對象的驗證資料精確度
        factor=0.5,                # 衰減學習率的比例
        patience=5,                # 監控對象的訓練週期次數
        verbose=1,                 # 降低學習率後執行通知
        mode='max',                # 監控最高數值 
        min_lr=0.0001)             # 學習率下限值 
    
    model = make_convlayer()
    callbacks_list = [reduce_lr]
    
    # 資料擴增 
    datagen = ImageDataGenerator(width_shift_range=0.1,   # 以圖像寬度的0.1比例隨機橫向移動 
                                 height_shift_range=0.1,  # 以圖像高度的0.1比例隨機上下移動
                                 rotation_range=10,       # 在10度的範圍內隨機旋轉
                                 zoom_range=0.1,          # 以原始尺寸的0.1比例隨機放大
                                 horizontal_flip=True)    # 左右翻轉

    # 小批次尺寸
    batch_size = 128
    # 訓練次數
    epochs = 100

    # 訓練
    history = model.fit(datagen.flow(x_train,# 看小批次的尺寸多少，就生成多少的增補資料
                                     y_train,
                                     batch_size=batch_size),
                        
                        steps_per_epoch=x_train.shape[0] // batch_size, # 每次訓練的步驟次數
                        epochs=epochs,                                  # 訓練次數
                        verbose=1,                                      # 輸出學習進度的情況
                        validation_data=(x_test, y_test),               # 驗證資料
                        callbacks=callbacks_list)

    return history


In [ ]:
x_train, x_test, y_train, y_test = prepare_data()

In [ ]:
%%time
history = train(x_train, x_test, y_train, y_test)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# 設定繪圖尺寸
plt.figure(figsize=(15, 10))
# 將圖形縮小、讓圖形之間保有空間
plt.subplots_adjust(wspace=0.2)

# 在2 × 1 格線的上方繪圖
plt.subplot(2, 1, 1)
# 繪製訓練資料與驗證資料的準確度
plt.plot(history.history['accuracy'], label='train', linestyle='--')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()              # 顯示圖例
plt.grid()                # 顯示格線
plt.xlabel('Epoch')       # x 軸標籤
plt.ylabel('Acc')         # y 軸標籤

# 在2 × 1 格線的下方繪圖
plt.subplot(2, 1, 2)
# 繪製學習率
plt.plot(history.history['lr'], label='Learning Rate')
plt.legend()                  # 顯示圖例
plt.grid()                    # 顯示格線
plt.xlabel('Epoch')           # x 軸標籤
plt.ylabel('Learning Rate')   # y 軸標籤
plt.show()
